## Importing modules

In [ ]:
import os
import time
import math
import string
import random

In [ ]:
import collections
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import torch
from torch import nn
from torch import optim
from torch.functional import F
from torch.utils.data import DataLoader

In [ ]:
from utils import evaluate
from utils import training

In [ ]:
from importlib import reload

## Loading data

In [ ]:
data_dict_re = torch.load("./saves/data/clean_names.pt")

In [ ]:
data_in_char = data_dict_re["data_in_char"]
char_vocab = data_dict_re["char_vocab"]

In [ ]:
print("data length:", len(data_in_char))
print("vocab size:", len(char_vocab))

In [ ]:
print(data_in_char[:5])

In [ ]:
char_to_ix = {ch:i for i,ch in enumerate(char_vocab)}
ix_to_char = {i:ch for ch,i in char_to_ix.items()}

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_as_str, _map, batched):
        self.batched = batched
        
        self.data_as_int = []
        self.max_seqlen = float("-inf")
        self.min_seqlen = float("inf")
        
        # Convert data to integers
        for seq_as_str in data_as_str:
            seq_as_int = evaluate.keys_to_values(seq_as_str, _map,
                random.choice(list(_map)))
            
            self.data_as_int.append(seq_as_int)
            self.max_seqlen = max(self.max_seqlen, len(seq_as_int)-1)
            self.min_seqlen = min(self.min_seqlen, len(seq_as_int)-1)

    def __len__(self):
        return len(self.data_as_int)

    def __getitem__(self, ix):
        # Get data sample at index, ix
        item = self.data_as_int[ix]
        PAD_ix = char_to_ix["<PAD>"]
        
        # Slice x and y from sample
        x_pad = item[:-1]
        x_len = len(x_pad)
        y_pad = item[ 1:]
        
        # Pad x and y to self.max_seqlen, if self.batched is True
        if self.batched:
            x_pad += ([PAD_ix] * (self.max_seqlen - len(x_pad)))
            y_pad += ([PAD_ix] * (self.max_seqlen - len(y_pad)))
            
        return (torch.tensor(x_pad), torch.tensor(x_len)), torch.tensor(y_pad)

In [ ]:
dataset = Dataset(data_in_char, char_to_ix, batched=True)
dataloader = DataLoader(dataset, 16, True)

In [ ]:
print("Dataset size:", len(dataset))
print("Max sequence length:", dataset.max_seqlen)
print("Min sequence length:", dataset.min_seqlen)

## Model

In [ ]:
class Model(nn.Module):
    def __init__(self, _map, hidden_size, emb_dim=8, n_layers=1, dropout_p=0.2):
        """
        Input:
            _map: char_to_ix.
            hidden_size: Number of features to learn.
            emb_dim: Size of embedding vector.
            n_layers: Number of layers.
            dropout_p: Dropout probability.
        """
        super(Model, self).__init__()
        
        self.vocab_size  = len(_map)
        self.hidden_size = hidden_size
        self.emb_dim     = emb_dim
        self.n_layers    = n_layers
        self.dropout_p   = dropout_p
        
        self.embedding = nn.Embedding(
            num_embeddings=self.vocab_size,
            embedding_dim =self.emb_dim,
            padding_idx   =_map["<PAD>"])
        
        self.lstm = nn.LSTM(
            input_size =self.emb_dim,
            hidden_size=self.hidden_size,
            num_layers =self.n_layers,
            batch_first=True)
        
        self.dropout = nn.Dropout(self.dropout_p)
        
        self.fc = nn.Linear(
            in_features =self.hidden_size,
            out_features=self.vocab_size)
        
    def forward(self, x, prev_state):
        """
        Input:
            x: (x, x_len)
            prev_state: The previous state.
            
        Output:
            out: The output of the model.
            state: The current state.
        """
        n_b, n_s = x[0].shape
        batched = True if (n_b>1 or len(x)>1) else False
        
        embed = self.embedding(x[0])
        
        # Pack-pad embeddings if x is batched
        if batched:
            embed = nn.utils.rnn.pack_padded_sequence(
                embed, x[1], True, False)
            
        yhat, state = self.lstm(embed, prev_state)
        
        # Un-pack-pad the lstm output
        if batched:
            yhat, _ = nn.utils.rnn.pad_packed_sequence(
                yhat, True, total_length=n_s)
            
        yhat = self.dropout(yhat)
        out = self.fc(yhat)
        return out, state
    
    def init_state(self, b_size=1):
        return (torch.zeros(self.n_layers, b_size, self.hidden_size),
                torch.zeros(self.n_layers, b_size, self.hidden_size))

In [ ]:
model = Model(char_to_ix, 64, 8, n_layers=1, dropout_p=0.2)
model

In [ ]:
loss_history = []

## Loading and Saving Model

In [ ]:
def load_model(path):
    m_data = torch.load(path)
    
    m = Model(
        _map       =m_data["_map"],
        hidden_size=m_data["hidden_size"],
        emb_dim    =m_data["emb_dim"],
        n_layers   =m_data["n_layers"],
        dropout_p  =m_data["dropout_p"])
    
    m.load_state_dict(m_data["state_dict"])
    l_hist = m_data["loss_history"]
    return m, l_hist, _map

**Uncomment cell to load the trained model**

In [ ]:
# model, losss_history = load_model("./saves/model/dino-name.pt")
# model

In [ ]:
def save_model(m, l_hist, _map, path=None):
    if not path: path = "./saves/model/dino-name_batch.pt"
        
    m_data = {
        "_map"        : _map,
        "hidden_size" : m.hidden_size,
        "emb_dim"     : m.emb_dim,
        "n_layers"    : m.n_layers,
        "dropout_p"   : m.dropout_p,
        "state_dict"  : m.state_dict(),
        "loss_history": l_hist}
    torch.save(m_data, path)

## Training

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=char_to_ix["<PAD>"])

In [ ]:
iteration = 250
per_iter = 50
start_t = time.time()

for _ti in range(iteration//per_iter):
    model, costs = training.train(
        model, dataloader, per_iter, criterion, clip=0.25, lr=1e-3, print_every=10,
        sleep=5, sleep_every=25)
    
    loss_history.extend(costs)
    save_model(model, loss_history, char_to_ix)
    time.sleep(2)
    
    print("\n" + "="*50)
    print("Round: {:2} of {:2}, Running Time: {:7.2f} sec".format(
        _ti+1, iteration//per_iter, time.time() - start_t))
    print("="*50 + "\n")

In [ ]:
cum = 5
plt.plot(
    [sum(loss_history[i:i+cum])/cum for i in range(0, len(loss_history), cum)])

In [ ]:
print("Iter: {} | Min: {:.4f} | Max: {:.4f} | Last: {:.4f} | Ave: {:.4f}".format(
    len(loss_history), min(loss_history), max(loss_history), loss_history[-1],
    sum(loss_history)/len(loss_history)))

## Evaluating

In [ ]:
n_samp = 100
ix_list = list(char_to_ix.values())[1:-1]

In [ ]:
originality = evaluate.originality(
    n_samp, dataset.data_as_int, evaluate.sample, model, ix_list,
    4, False, 30, char_to_ix["<EOS>"])

print("Duplicates: {} of {}".format(len(originality[1]), n_samp))
print("{:6.2f}% recall".format(100 * len(originality[1]) / n_samp))
print("{:6.2f}% original".format(100 - (100 * len(originality[1]) / n_samp)))

In [ ]:
for i in range(10):
    print("".join(evaluate.keys_to_values(
        evaluate.sample(
            model, random.choice(ix_list), 5, False, 30, char_to_ix["<EOS>"]),
        ix_to_char, "<?>")))

In [ ]:
for i in range(10):
    print("".join(evaluate.keys_to_values(
        evaluate.sample(
            model, evaluate.keys_to_values(
                list("giant"), char_to_ix, char_to_ix["<PAD>"]
            ), 5, False, 30, char_to_ix["<EOS>"]),
        ix_to_char, "<?>")))

In [ ]:
for ch in char_vocab:
    print("".join(evaluate.keys_to_values(
        evaluate.sample(
            model, char_to_ix[ch], 1, True, 30, char_to_ix["<EOS>"]),
        ix_to_char, "<?>")))